In [1]:
import pandas as pd
import numpy as np
import os 
import scipy.stats as st
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns
from get_text import case_names, text, get_df, get_all_df

In [2]:
# Import the master dataframes; one will include stemmed speech while the other will not
df = get_all_df(stemmed='stemmed')
df_no_stem = get_all_df(stemmed='notstemmed')

In [3]:
# script for obtaining the dictionaries for mapping the key:value pairs in the dataframe 
import bs4 as bs
import urllib.request  
import requests
from urllib.parse import urljoin

def get_codes(input_item):

    url_get = ('http://scdb.wustl.edu/documentation.php?var=' + str(input_item))
    sauce = urllib.request.urlopen(str(url_get)).read()
    soup = bs.BeautifulSoup(sauce, 'lxml')

    things = []
    for a in soup.find_all('td'):
        x = a.string
        y = str(x).strip()
        things.append(y)

    stop = (len(things)-15)
    things = things[23:stop]
    new_keys = dict(zip(things[::2], things[1::2]))
    new_keys_two = {int(float(key)):new_keys[key] for key in new_keys}
    return new_keys_two

In [4]:
# Create a master dictionary of these key:value pair mappings 

codes = ['issueArea', 'petitionerState', 'petitioner', 'decisionDirection', 'decisionDirectionDissent',
        'authorityDecision2', 'caseOrigin', 'caseOriginState', 'chief', 'lawType', 'lawSupp', 'declarationUncon',
        'caseDisposition', 'partyWinning', 'caseDisposition', 'precedentAlteration', 'vote', 'opinion', 'direction', 
        'majority']

master_dict = {}

for name in codes:
    master_dict['%s' % name] = get_codes(name)

In [5]:
df_no_stem['issueAreas'] = df_no_stem['issueArea'].map(master_dict['issueArea'])
df_no_stem.groupby(['issueAreas']).count().issueArea.sort_values(ascending=False)

issueAreas
Criminal Procedure    218
Economic Activity     159
Civil Rights          134
Judicial Power        106
First Amendment        42
Federalism             40
Due Process            25
Privacy                15
Unions                 14
Attorneys              13
Federal Taxation        9
Miscellaneous           4
Private Action          1
Name: issueArea, dtype: int64

In [6]:
# Isolating Kennedy's Text by Issue Areas with at least 40 cases 

criminal_procedure = df_no_stem[df_no_stem['issueAreas'] == 'Criminal Procedure']
econ_activity = df_no_stem[df_no_stem['issueAreas'] == 'Economic Activity']
civil_rights = df_no_stem[df_no_stem['issueAreas'] == 'Civil Rights']
judicial_power = df_no_stem[df_no_stem['issueAreas'] == 'Judicial Power']
first_amendment = df_no_stem[df_no_stem['issueAreas'] == 'First Amendment']
federalism = df_no_stem[df_no_stem['issueAreas'] == 'Federalism']

In [7]:
# Keeps rows where winner has a value; drop NaN values

criminal_procedure = criminal_procedure[np.isfinite(criminal_procedure['winner'])]
econ_activity = econ_activity[np.isfinite(econ_activity['winner'])]
civil_rights = civil_rights[np.isfinite(civil_rights['winner'])]
judicial_power = judicial_power[np.isfinite(judicial_power['winner'])]
first_amendment = first_amendment[np.isfinite(first_amendment['winner'])]
federalism = federalism[np.isfinite(federalism['winner'])]

In [8]:
# Import Transformers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Import Model Selectors - Hyperparameter Optimizers 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# Import Estimators 
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

# Import Pipeline 
from sklearn.pipeline import Pipeline

# Import NLTK
import nltk

# Import Accuracy Score for MNB
from sklearn.metrics import accuracy_score

from sklearn.model_selection import StratifiedKFold

In [9]:
# Split train and test data for every issue area 

from sklearn.model_selection import train_test_split

# Criminal Law 
crim_features_train, crim_features_test, crim_labels_train, crim_labels_test = train_test_split(
    criminal_procedure.speech, criminal_procedure.winner.values, test_size = 0.5, random_state = 42)

# Economic Activity
econ_features_train, econ_features_test, econ_labels_train, econ_labels_test = train_test_split(
    econ_activity.speech, econ_activity.winner.values, test_size = 0.5, random_state = 42)

# Civil Rights 
cr_features_train, cr_features_test, cr_labels_train, cr_labels_test = train_test_split(
    civil_rights.speech, civil_rights.winner.values, test_size = 0.5, random_state = 42)

# Judicial Power 
jp_features_train, jp_features_test, jp_labels_train, jp_labels_test = train_test_split(
    judicial_power.speech, judicial_power.winner.values, test_size = 0.5, random_state = 42)

# First Amendment 
famd_features_train, famd_features_test, famd_labels_train, famd_labels_test = train_test_split(
    first_amendment.speech, first_amendment.winner.values, test_size = 0.5, random_state = 42)

# Federalism 
federalism_features_train, federalism_features_test, federalism_labels_train, federalism_labels_test = train_test_split(
    federalism.speech, federalism.winner.values, test_size = 0.5, random_state = 42)

#### Naive Bayes Classifer on the Data

In [10]:
count_vec = CountVectorizer()
tf_idf = TfidfTransformer()
clf_MNB = MultinomialNB()

steps = [('count_vec', count_vec),
         ('tf_idf', tf_idf),
         ('clf_MNB', clf_MNB)]

p_grid = [{'count_vec__min_df': range(1,5)}]

pipeline = Pipeline(steps)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
nb = GridSearchCV(pipeline, param_grid=p_grid, cv=skf)

In [11]:
# Fitting the Naive Bayes model with cases involving Criminal Procedure 

nb.fit(crim_features_train, crim_labels_train)
crim_labels_pred = nb.best_estimator_.predict(crim_features_test)
print(accuracy_score(crim_labels_test, crim_labels_pred))
print(nb.best_params_)

0.688073394495
{'count_vec__min_df': 3}


In [12]:
# Fitting the Naive Bayes model with cases involving Economic Activity  

nb.fit(econ_features_train, econ_labels_train)
econ_labels_pred = nb.best_estimator_.predict(econ_features_test)
print(accuracy_score(econ_labels_test, econ_labels_pred))
print(nb.best_params_)

0.772151898734
{'count_vec__min_df': 1}


In [13]:
# Fitting the Naive Bayes model with cases involving Civil Rights 

nb.fit(cr_features_train, cr_labels_train)
cr_labels_pred = nb.best_estimator_.predict(cr_features_test)
print(accuracy_score(cr_labels_test, cr_labels_pred))
print(nb.best_params_)

0.815384615385
{'count_vec__min_df': 4}


In [14]:
# Fitting the Naive Bayes model with cases involving Judicial Power 

nb.fit(jp_features_train, jp_labels_train)
jp_labels_pred = nb.best_estimator_.predict(jp_features_test)
print(accuracy_score(jp_labels_test, jp_labels_pred))
print(nb.best_params_)

0.528301886792
{'count_vec__min_df': 1}


In [15]:
# Fitting the Naive Bayes model with cases involving First Amendment  

nb.fit(famd_features_train, famd_labels_train)
famd_labels_pred = nb.best_estimator_.predict(famd_features_test)
print(accuracy_score(famd_labels_test, famd_labels_pred))
print(nb.best_params_)

0.666666666667
{'count_vec__min_df': 1}


In [16]:
# Fitting the Naive Bayes model with cases involving Federalism

nb.fit(federalism_features_train, federalism_labels_train)
federalism_labels_pred = nb.best_estimator_.predict(federalism_features_test)
print(accuracy_score(federalism_labels_test, federalism_labels_pred))
print(nb.best_params_)

0.45
{'count_vec__min_df': 1}


In [17]:
naive = {"Criminal Procedure": accuracy_score(crim_labels_test, crim_labels_pred),
         "Economic Activity": accuracy_score(econ_labels_test, econ_labels_pred),
         "Civil Rights": accuracy_score(cr_labels_test, cr_labels_pred),
         "Judicial Power": accuracy_score(jp_labels_test, jp_labels_pred),
         "First Amendment": accuracy_score(famd_labels_test, famd_labels_pred),
         "Federalism": accuracy_score(federalism_labels_test, federalism_labels_pred)}

In [18]:
naive

{'Civil Rights': 0.81538461538461537,
 'Criminal Procedure': 0.68807339449541283,
 'Economic Activity': 0.77215189873417722,
 'Federalism': 0.45000000000000001,
 'First Amendment': 0.66666666666666663,
 'Judicial Power': 0.52830188679245282}

In [19]:
naive2 = pd.DataFrame(naive, index=['accuracy'])
naive2 = naive2.unstack().unstack()

#### SVM Classifer on the Data

In [20]:
count_vec = CountVectorizer()
tf_idf = TfidfTransformer()
clf_SVC = SVC()

steps = [('count_vec', count_vec),
         ('tf_idf', tf_idf),
         ('clf_SVC', clf_SVC)]

p_grid = [{'count_vec__min_df': range(1,5),
           'clf_SVC__random_state': [42],
           'clf_SVC__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
           'clf_SVC__C': [0.1,1,10,100,1000],
           'clf_SVC__gamma': [0.1,1,10,100,1000]}]

pipeline = Pipeline(steps)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
gs = GridSearchCV(pipeline, param_grid=p_grid, cv=skf)

In [21]:
# Fitting the SVM model with cases involving Criminal Procedure

gs.fit(crim_features_train, crim_labels_train)
print(gs.best_params_)
print(gs.best_estimator_.score(crim_features_test, crim_labels_test))

{'clf_SVC__C': 1, 'clf_SVC__gamma': 0.1, 'clf_SVC__kernel': 'linear', 'clf_SVC__random_state': 42, 'count_vec__min_df': 1}
0.642201834862


In [22]:
# Fitting the SVM model with cases involving Economic Activity

gs.fit(econ_features_train, econ_labels_train)
print(gs.best_params_)
print(gs.best_estimator_.score(econ_features_test, econ_labels_test))

{'clf_SVC__C': 0.1, 'clf_SVC__gamma': 0.1, 'clf_SVC__kernel': 'linear', 'clf_SVC__random_state': 42, 'count_vec__min_df': 1}
0.772151898734


In [23]:
# Fitting the SVM model with cases involving Civil Rights 

gs.fit(cr_features_train, cr_labels_train)
print(gs.best_params_)
print(gs.best_estimator_.score(cr_features_test, cr_labels_test))

{'clf_SVC__C': 10, 'clf_SVC__gamma': 0.1, 'clf_SVC__kernel': 'rbf', 'clf_SVC__random_state': 42, 'count_vec__min_df': 4}
0.769230769231


In [24]:
# Fitting the SVM model with cases involving Judicial Power 

gs.fit(jp_features_train, jp_labels_train)
print(gs.best_params_)
print(gs.best_estimator_.score(jp_features_test, jp_labels_test))

{'clf_SVC__C': 10, 'clf_SVC__gamma': 1, 'clf_SVC__kernel': 'sigmoid', 'clf_SVC__random_state': 42, 'count_vec__min_df': 4}
0.528301886792


In [25]:
# Fitting the SVM model with cases involving First Amendment 

gs.fit(famd_features_train, famd_labels_train)
print(gs.best_params_)
print(gs.best_estimator_.score(famd_features_test, famd_labels_test))

{'clf_SVC__C': 10, 'clf_SVC__gamma': 10, 'clf_SVC__kernel': 'sigmoid', 'clf_SVC__random_state': 42, 'count_vec__min_df': 4}
0.52380952381


In [26]:
# Fitting the SVM model with cases involving Federalism 

gs.fit(federalism_features_train, federalism_labels_train)
print(gs.best_params_)
print(gs.best_estimator_.score(federalism_features_test, federalism_labels_test))

{'clf_SVC__C': 0.1, 'clf_SVC__gamma': 0.1, 'clf_SVC__kernel': 'linear', 'clf_SVC__random_state': 42, 'count_vec__min_df': 1}
0.45


In [27]:
svm = {"Criminal Procedure": gs.best_estimator_.score(crim_features_test, crim_labels_test),
         "Economic Activity": gs.best_estimator_.score(econ_features_test, econ_labels_test),
         "Civil Rights": gs.best_estimator_.score(cr_features_test, cr_labels_test),
         "Judicial Power": gs.best_estimator_.score(jp_features_test, jp_labels_test),
         "First Amendment": gs.best_estimator_.score(famd_features_test, famd_labels_test),
         "Federalism": gs.best_estimator_.score(federalism_features_test, federalism_labels_test)}

In [28]:
svm2 = pd.DataFrame(svm, index=['accuracy'])
svm2 = svm2.unstack().unstack()

#### Decision Tree Classifer on the Data

In [29]:
# Testing the Decision Tree Classifier on this data 

clf_DTC = DecisionTreeClassifier()

steps = [('counts', CountVectorizer()),
         ('tf_idf', TfidfTransformer()),
         ('clf_DTC', DecisionTreeClassifier())]

pipeline = Pipeline(steps)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
p_grid = [{'counts__min_df':[1],
           'clf_DTC__random_state': [42],
           'clf_DTC__min_samples_leaf': range(1,30),
           'clf_DTC__min_samples_split': range(2,30)}]

gs_one = GridSearchCV(pipeline, param_grid=p_grid, cv=skf)

In [31]:
# Fitting the Decision Tree Classifier model with cases involving Criminal Procedure 

gs_one.fit(crim_features_train, crim_labels_train)
print(gs_one.best_params_)
print(gs_one.best_estimator_.score(crim_features_test, crim_labels_test))

{'clf_DTC__min_samples_leaf': 14, 'clf_DTC__min_samples_split': 2, 'clf_DTC__random_state': 42, 'counts__min_df': 1}
0.522935779817


In [32]:
# Fitting the Decision Tree Classifier model with cases involving Economic Activity

gs_one.fit(econ_features_train, econ_labels_train)
print(gs_one.best_params_)
print(gs_one.best_estimator_.score(econ_features_test, econ_labels_test))

{'clf_DTC__min_samples_leaf': 17, 'clf_DTC__min_samples_split': 2, 'clf_DTC__random_state': 42, 'counts__min_df': 1}
0.582278481013


In [33]:
# Fitting the Decision Tree Classifier model with cases involving Civil Rights 

gs_one.fit(cr_features_train, cr_labels_train)
print(gs_one.best_params_)
print(gs_one.best_estimator_.score(cr_features_test, cr_labels_test))

{'clf_DTC__min_samples_leaf': 3, 'clf_DTC__min_samples_split': 2, 'clf_DTC__random_state': 42, 'counts__min_df': 1}
0.707692307692


In [34]:
# Fitting the Decision Tree Classifier model with cases involving Judicial Power

gs_one.fit(jp_features_train, jp_labels_train)
print(gs_one.best_params_)
print(gs_one.best_estimator_.score(jp_features_test, jp_labels_test))

{'clf_DTC__min_samples_leaf': 1, 'clf_DTC__min_samples_split': 2, 'clf_DTC__random_state': 42, 'counts__min_df': 1}
0.528301886792


In [35]:
# Fitting the Decision Tree Classifier model with cases involving First Amendment

gs_one.fit(famd_features_train, famd_labels_train)
print(gs_one.best_params_)
print(gs_one.best_estimator_.score(famd_features_test, famd_labels_test))

{'clf_DTC__min_samples_leaf': 3, 'clf_DTC__min_samples_split': 2, 'clf_DTC__random_state': 42, 'counts__min_df': 1}
0.666666666667


In [36]:
# Fitting the Decision Tree Classifier model with cases involving Federalism

gs_one.fit(federalism_features_train, federalism_labels_train)
print(gs_one.best_params_)
print(gs_one.best_estimator_.score(federalism_features_test, federalism_labels_test))

{'clf_DTC__min_samples_leaf': 1, 'clf_DTC__min_samples_split': 16, 'clf_DTC__random_state': 42, 'counts__min_df': 1}
0.5


In [37]:
dtc = {"Criminal Procedure": gs_one.best_estimator_.score(crim_features_test, crim_labels_test),
         "Economic Activity": gs_one.best_estimator_.score(econ_features_test, econ_labels_test),
         "Civil Rights": gs_one.best_estimator_.score(cr_features_test, cr_labels_test),
         "Judicial Power": gs_one.best_estimator_.score(jp_features_test, jp_labels_test),
         "First Amendment": gs_one.best_estimator_.score(famd_features_test, famd_labels_test),
         "Federalism": gs_one.best_estimator_.score(federalism_features_test, federalism_labels_test)}

In [38]:
dtc2 = pd.DataFrame(dtc, index=['accuracy'])
dtc2 = dtc2.unstack().unstack()

#### AdaBoost Classifer on the Data

In [39]:
# Testing the AdaBoost Classifier on this data 

clf_ada = AdaBoostClassifier()

steps = [('counts', CountVectorizer()),
         ('tf_idf', TfidfTransformer()),
         ('clf_ada', AdaBoostClassifier())]

pipeline = Pipeline(steps)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
p_grid = [{'counts__min_df':[1],
           'clf_ada__random_state': [42],
           'clf_ada__n_estimators': range(1,30)}]

gs_two = GridSearchCV(pipeline, param_grid=p_grid, cv=skf)

In [40]:
# Fitting the AdaBoost Classifier model with cases involving Criminal Procedure

gs_two.fit(crim_features_train, crim_labels_train)
print(gs_two.best_params_)
print(gs_two.best_estimator_.score(crim_features_test, crim_labels_test))

{'clf_ada__n_estimators': 5, 'clf_ada__random_state': 42, 'counts__min_df': 1}
0.51376146789


In [41]:
# Fitting the AdaBoost Classifier model with cases involving Economic Activity

gs_two.fit(econ_features_train, econ_labels_train)
print(gs_two.best_params_)
print(gs_two.best_estimator_.score(econ_features_test, econ_labels_test))

{'clf_ada__n_estimators': 23, 'clf_ada__random_state': 42, 'counts__min_df': 1}
0.594936708861


In [42]:
# Fitting the AdaBoost Classifier model with cases involving Civil Rights 

gs_two.fit(cr_features_train, cr_labels_train)
print(gs_two.best_params_)
print(gs_two.best_estimator_.score(cr_features_test, cr_labels_test))

{'clf_ada__n_estimators': 1, 'clf_ada__random_state': 42, 'counts__min_df': 1}
0.769230769231


In [43]:
# Fitting the AdaBoost Classifier model with cases involving Judicial Power

gs_two.fit(jp_features_train, jp_labels_train)
print(gs_two.best_params_)
print(gs_two.best_estimator_.score(jp_features_test, jp_labels_test))

{'clf_ada__n_estimators': 1, 'clf_ada__random_state': 42, 'counts__min_df': 1}
0.528301886792


In [44]:
# Fitting the AdaBoost Classifier model with cases involving First Amendment

gs_two.fit(famd_features_train, famd_labels_train)
print(gs_two.best_params_)
print(gs_two.best_estimator_.score(famd_features_test, famd_labels_test))

{'clf_ada__n_estimators': 1, 'clf_ada__random_state': 42, 'counts__min_df': 1}
0.666666666667


In [45]:
# Fitting the AdaBoost Classifier model with cases involving Federalism

gs_two.fit(federalism_features_train, federalism_labels_train)
print(gs_two.best_params_)
print(gs_two.best_estimator_.score(federalism_features_test, federalism_labels_test))

{'clf_ada__n_estimators': 1, 'clf_ada__random_state': 42, 'counts__min_df': 1}
0.5


In [46]:
ada = {"Criminal Procedure": gs_two.best_estimator_.score(crim_features_test, crim_labels_test),
         "Economic Activity": gs_two.best_estimator_.score(econ_features_test, econ_labels_test),
         "Civil Rights": gs_two.best_estimator_.score(cr_features_test, cr_labels_test),
         "Judicial Power": gs_two.best_estimator_.score(jp_features_test, jp_labels_test),
         "First Amendment": gs_two.best_estimator_.score(famd_features_test, famd_labels_test),
         "Federalism": gs_two.best_estimator_.score(federalism_features_test, federalism_labels_test)}

In [47]:
ada2 = pd.DataFrame(ada, index=['accuracy'])
ada2 = ada2.unstack().unstack()

#### Random Forest Classifer on the Data

In [48]:
# Testing the Random Forest Classifier on this data 

clf_RFC = RandomForestClassifier()

steps = [('counts', CountVectorizer()),
         ('tf_idf', TfidfTransformer()),
         ('clf_RFC', RandomForestClassifier())]

pipeline = Pipeline(steps)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
p_grid = [{'counts__min_df':[1],
           'clf_RFC__random_state': [42],
           'clf_RFC__n_estimators': [15,20,25,30],
           'clf_RFC__min_samples_leaf': range(1,30),
           'clf_RFC__min_samples_split': range(2,30)}]

gs_three = GridSearchCV(pipeline, param_grid=p_grid, cv=skf)

In [49]:
# Fitting the Random Forest Classifier model with cases involving Criminal Procedure

gs_three.fit(crim_features_train, crim_labels_train)
print(gs_three.best_params_)
print(gs_three.best_estimator_.score(crim_features_test, crim_labels_test))

{'clf_RFC__min_samples_leaf': 1, 'clf_RFC__min_samples_split': 7, 'clf_RFC__n_estimators': 15, 'clf_RFC__random_state': 42, 'counts__min_df': 1}
0.669724770642


In [50]:
# Fitting the Random Forest Classifier model with cases involving Economic Activity

gs_three.fit(econ_features_train, econ_labels_train)
print(gs_three.best_params_)
print(gs_three.best_estimator_.score(econ_features_test, econ_labels_test))

{'clf_RFC__min_samples_leaf': 2, 'clf_RFC__min_samples_split': 5, 'clf_RFC__n_estimators': 15, 'clf_RFC__random_state': 42, 'counts__min_df': 1}
0.721518987342


In [51]:
# Fitting the Random Forest Classifier model with cases involving Civil Rights 

gs_three.fit(cr_features_train, cr_labels_train)
print(gs_three.best_params_)
print(gs_three.best_estimator_.score(cr_features_test, cr_labels_test))

{'clf_RFC__min_samples_leaf': 1, 'clf_RFC__min_samples_split': 19, 'clf_RFC__n_estimators': 30, 'clf_RFC__random_state': 42, 'counts__min_df': 1}
0.784615384615


In [52]:
# Fitting the Random Forest Classifier model with cases involving Judicial Power

gs_three.fit(jp_features_train, jp_labels_train)
print(gs_three.best_params_)
print(gs_three.best_estimator_.score(jp_features_test, jp_labels_test))

{'clf_RFC__min_samples_leaf': 1, 'clf_RFC__min_samples_split': 2, 'clf_RFC__n_estimators': 15, 'clf_RFC__random_state': 42, 'counts__min_df': 1}
0.547169811321


In [53]:
# Fitting the Random Forest Classifier model with cases involving First Amendment

gs_three.fit(famd_features_train, famd_labels_train)
print(gs_three.best_params_)
print(gs_three.best_estimator_.score(famd_features_test, famd_labels_test))

{'clf_RFC__min_samples_leaf': 1, 'clf_RFC__min_samples_split': 2, 'clf_RFC__n_estimators': 15, 'clf_RFC__random_state': 42, 'counts__min_df': 1}
0.666666666667


In [54]:
# Fitting the Random Forest Classifier model with cases involving Federalism

gs_three.fit(federalism_features_train, federalism_labels_train)
print(gs_three.best_params_)
print(gs_three.best_estimator_.score(federalism_features_test, federalism_labels_test))

{'clf_RFC__min_samples_leaf': 1, 'clf_RFC__min_samples_split': 3, 'clf_RFC__n_estimators': 30, 'clf_RFC__random_state': 42, 'counts__min_df': 1}
0.45


In [55]:
rfc = {"Criminal Procedure": gs_three.best_estimator_.score(crim_features_test, crim_labels_test),
         "Economic Activity": gs_three.best_estimator_.score(econ_features_test, econ_labels_test),
         "Civil Rights": gs_three.best_estimator_.score(cr_features_test, cr_labels_test),
         "Judicial Power": gs_three.best_estimator_.score(jp_features_test, jp_labels_test),
         "First Amendment": gs_three.best_estimator_.score(famd_features_test, famd_labels_test),
         "Federalism": gs_three.best_estimator_.score(federalism_features_test, federalism_labels_test)}

In [56]:
rfc2 = pd.DataFrame(rfc, index=['accuracy'])
rfc2 = rfc2.unstack().unstack()